Treinaremos nosso modelo ajustando os pesos várias vezes para fazer previsões cada vez melhores, utilizando uma técnica de otimização chamada gradiente de descida.

## Dados de treinamento

In [1]:
import numpy as np
import torch

Podemos representar os dados de treinamento utilizando duas matrizes: `inputs` e `targets`, cada uma com uma linha por observação e uma coluna por variável.

In [2]:
# Input (temperatura, chuva, umidade)
inputs = np.array(
    [[73, 67, 43], [91, 88, 64], [87, 134, 58], [102, 43, 37], [69, 96, 70]],
    dtype="float32",
)

In [3]:
# Targets (maçãs, laranjas)
targets = np.array(
    [[56, 70], [81, 101], [119, 133], [22, 37], [103, 119]], dtype="float32"
)

In [4]:
# Converte os inputs e targets para tensores
inputs_t = torch.from_numpy(inputs)
targets_t = torch.from_numpy(targets)

print(inputs)
print(targets)

[[ 73.  67.  43.]
 [ 91.  88.  64.]
 [ 87. 134.  58.]
 [102.  43.  37.]
 [ 69.  96.  70.]]
[[ 56.  70.]
 [ 81. 101.]
 [119. 133.]
 [ 22.  37.]
 [103. 119.]]


Os pesos e os viéses também podem ser representados por matrizes, inicializados como valores aleatórios por `randn`. A primeira linha de `w` e o primeiro elemento de `b` são usados para predizer a primeira variável-alvo.

In [5]:
# Pesos e bias
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[ 1.7345, -0.5321,  1.3262],
        [ 0.6187, -0.5966,  1.5623]], requires_grad=True)
tensor([ 0.4502, -0.4211], requires_grad=True)


In [6]:
def model(x):
    return x @ w.t() + b

In [7]:
model(inputs_t)

tensor([[148.4476,  71.9531],
        [196.3454, 103.3699],
        [156.9734,  64.0800],
        [203.5626,  94.8393],
        [161.8858,  94.3595]], grad_fn=<AddBackward0>)

A matriz obtida pela operação `x @ w.t() + b` é a predição do modelo para as variáveis alvos 

In [8]:
# Gerando previsões
preds = model(inputs_t)
print(preds)

tensor([[148.4476,  71.9531],
        [196.3454, 103.3699],
        [156.9734,  64.0800],
        [203.5626,  94.8393],
        [161.8858,  94.3595]], grad_fn=<AddBackward0>)


In [9]:
# Comparando com os targets
print(targets_t)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


## Função de perda

Agora que temos um modelo, podemos melhorá-lo.

Precisamos, antes desse passo, encontrar uma forma de saber se a predição foi além ou de menos. Podemos comparar o alvo com a previsão realizada com os seguintes meios:

* Calculando a diferença entre as duas matrizes
* Elevar todos os elementos ao quadrado para remover os valores negativos
* Calcular a média dos elementos resultantes

O resultado é um único valor, conhecido como MSE (mean square error) -- Erro Quadrático Médio

In [10]:
# Erro quadrático médio
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [11]:
# Calculando o erro
loss = mse(preds, targets_t)
print(loss)

tensor(6843.7578, grad_fn=<DivBackward0>)


In [12]:
# Gradientes
print(f'Gradiente de loss: {loss.backward()}')
print(f'Gradiente de w: {w.grad}')

Gradiente de loss: None
Gradiente de w: tensor([[ 8626.2607,  6978.6113,  4879.9272],
        [ -287.6768, -1754.8534,  -669.2966]])


## Ajustando pesos e bias para reduzir a perda

O objetivo de todo treinamento de modelo é diminuir a perda.

Se o grandiente é positivo: 

* aumentar o valor do peso, aumentará o valor da perda.
* diminuir o valor do peso, diminuirá o valor da perda

In [13]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

In [14]:
# Verificamos se as perdas diminuiram
preds = model(inputs_t)
loss = mse(preds, targets_t)
print(loss)

tensor(5470.1572, grad_fn=<DivBackward0>)


In [15]:
w.grad.zero_()
b.grad.zero_()

print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


## Treinamento utilizando descida do gradiente

Como visto, reduzimos a perda para melhorar o modelo usando algoritmos de descida de gradiente. Para treinar modelos seguindo essa estratégia, precisamos:

1. Gerar predições
2. Calcular a perda
3. Calcular os gradientes
4. Ajustar os pesos substraindo por uma pequena fração do gradiente
5. Redefinir os gradientes a zero

In [16]:
# Geramos predições
preds = model(inputs_t)
print(preds)

tensor([[135.3754,  73.6267],
        [179.2302, 105.6044],
        [137.2859,  67.0701],
        [189.9565,  96.1351],
        [145.8173,  96.7112]], grad_fn=<AddBackward0>)


In [17]:
# Calculamos a perda
loss = mse(preds, targets_t)
print(loss)

tensor(5470.1572, grad_fn=<DivBackward0>)


In [18]:
# Computamos os gradientes
loss.backward()
print(w.grad)
print(b.grad)

tensor([[ 7282.0376,  5549.0630,  3994.4126],
        [ -111.6606, -1556.6702,  -549.1049]])
tensor([81.3331, -4.1705])


In [19]:
# Ajustamos o peso
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

    w.grad.zero_()
    b.grad.zero_()

In [20]:
print(w)
print(b)

tensor([[ 1.5755, -0.6574,  1.2375],
        [ 0.6227, -0.5634,  1.5745]], requires_grad=True)
tensor([ 0.4484, -0.4210], requires_grad=True)


In [21]:
# Calculamos a perda
preds = model(inputs_t)
loss = mse(preds, targets_t)
print(loss)

tensor(4533.8604, grad_fn=<DivBackward0>)


## Treinamento por épocas

Para reduzirmos ainda mais a perda, podemos repetir esse processo por várias vezes (épocas). Vamos treinar por 100 épocas

In [22]:
# Treinamento por 100 epochs
for i in range(800):
    preds = model(inputs_t)
    loss = mse(preds, targets_t)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [23]:
# Calculamos a perda
preds = model(inputs_t)
loss = mse(preds, targets_t)
print(loss)

tensor(12.7314, grad_fn=<DivBackward0>)


In [24]:
# Verificamos as predições com os targets
preds, targets

# Registrando o loss de 8.6:
# (tensor([[ 57.3090,  69.2779],
#          [ 82.1458,  99.8025],
#          [118.6023, 136.5943],
#          [ 21.4513,  29.5221],
#          [101.6896, 122.3019]], grad_fn=<AddBackward0>),
#  array([[ 56.,  70.],
#         [ 81., 101.],
#         [119., 133.],
#         [ 22.,  37.],
#         [103., 119.]], dtype=float32))

(tensor([[ 57.1209,  69.9705],
         [ 84.2266, 103.6373],
         [114.1721, 126.7533],
         [ 20.6127,  35.7886],
         [105.7925, 124.9305]], grad_fn=<AddBackward0>),
 array([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]], dtype=float32))

# Regressão linear utilizando ferramentas do PyTorch

In [25]:
import torch.nn as nn

In [26]:
# Inputs (temperatura, chuva, umidade)
inputs = np.array(
    [
        [73, 67, 43],
        [91, 88, 64],
        [87, 134, 58],
        [102, 43, 37],
        [69, 96, 70],
        [74, 66, 43],
        [91, 87, 65],
        [88, 134, 59],
        [101, 44, 37],
        [68, 96, 71],
        [73, 66, 44],
        [92, 87, 64],
        [87, 135, 57],
        [103, 43, 36],
        [68, 97, 70],
    ],
    dtype="float32",
)

# Targets (maçãs, laranjas)
targets = np.array(
    [
        [56, 70],
        [81, 101],
        [119, 133],
        [22, 37],
        [103, 119],
        [57, 69],
        [80, 102],
        [118, 132],
        [21, 38],
        [104, 118],
        [57, 69],
        [82, 100],
        [118, 134],
        [20, 38],
        [102, 120],
    ],
    dtype="float32",
)

In [27]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [28]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

In [29]:
from torch.utils.data import TensorDataset

In [30]:
# Definimos o dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [31]:
# Definimos o DataLoader resposável por gerar os batches
from torch.utils.data import DataLoader

In [32]:
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

`Shuffle` ajuda a randomizar o lote antes de passar para o modelo. Ele ajuda a melhorar a eficiência do algoritmo de otimização e leva a uma melhora acelerada da perda.

In [33]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[91., 87., 65.],
        [74., 66., 43.],
        [69., 96., 70.],
        [73., 66., 44.],
        [73., 67., 43.]])
tensor([[ 80., 102.],
        [ 57.,  69.],
        [103., 119.],
        [ 57.,  69.],
        [ 56.,  70.]])


In [34]:
# Definimos o modelo
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.5283, -0.2091, -0.2163],
        [-0.0149,  0.3261, -0.4483]], requires_grad=True)
Parameter containing:
tensor([ 0.0714, -0.4278], requires_grad=True)


O método `.parameters` retorna uma lista das matrizes dos pesos e do bias no modelo.

In [35]:
# Parametros
list(model.parameters())

[Parameter containing:
 tensor([[-0.5283, -0.2091, -0.2163],
         [-0.0149,  0.3261, -0.4483]], requires_grad=True),
 Parameter containing:
 tensor([ 0.0714, -0.4278], requires_grad=True)]

In [36]:
# Geramos as predições
preds = model(inputs)
preds

tensor([[-61.8041,   1.0621],
        [-80.2466,  -1.7705],
        [-86.4564,  15.9806],
        [-70.8064,  -4.5063],
        [-71.5958,  -1.5242],
        [-62.1232,   0.7211],
        [-80.2538,  -2.5449],
        [-87.2010,  15.5175],
        [-70.4873,  -4.1654],
        [-71.2839,  -1.9576],
        [-61.8112,   0.2877],
        [-80.5657,  -2.1115],
        [-86.4493,  16.7550],
        [-71.1184,  -4.0729],
        [-71.2767,  -1.1832]], grad_fn=<AddmmBackward0>)

## Função de perda

In [37]:
import torch.nn.functional as F

In [38]:
# Definimos a função de perda
loss_fn = F.mse_loss

In [39]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(16628.8535, grad_fn=<MseLossBackward0>)


## Otimizador

Podemos utilizar otimizadores embutidos no PyTorch em vez de definir manualmente. Para o gradiente de descida, podemos utilizar `optim.SGD`

In [40]:
# Definimos o otimizador
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

In [42]:
# Definimos a função de treinamento
def fit(num_epochs, model, loss_fn, opt):
    for epoch in range(num_epochs):
        for xb, yb in train_dl:
            # Geramos as predições
            pred = model(xb)
            # Calculamos a perda
            loss = loss_fn(pred, yb)
            # Computamos os gradientes
            loss.backward()
            # Atualizamos os parâmetros usando os gradientes
            opt.step()
            # Zeramos os gradientes
            opt.zero_grad()
        # Imprimimos a perda a cada 100 epochs
        if (epoch + 1) % 100 == 0:
            print(
                "Epoch [{}/{}], Loss: {:.4f}".format(epoch + 1, num_epochs, loss.item())
            )